In [ ]:
import pandas as pd
import numpy as np
import datetime
from pandas_datareader import data as wb
import statsmodels.api as sm

In [ ]:
def price_divs_book_value(work_table):
    print('Стоимость акции по дивам и балансовой стоимости (наивный метод)')

    try:
        dvd_avg = np.mean((work_table['Dividend per Share']).astype(float))  # Определим средний дивиденд
        # print(dvd_avg)
    except:
        dvd_avg = 0

    # risk_free_rate = 0.0179
    start = datetime.datetime(1999, 12, 31)
    end = datetime.datetime.today().strftime('%Y-%m-%d')
    Treasury = wb.DataReader(['DGS10'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    risk_free_rate = RF / 100
    # The discounted capital for the dividend of 3-year later
    dvd_avg / (1 + risk_free_rate) ** 3
    # The discounted capital for the dividend of 10-year later
    # dvd_avg/(1+risk_free_rate)**10

    dvd_total = 0
    for j in range(1, 11):
        dvd_total = dvd_total + dvd_avg / (1 + risk_free_rate) ** j
    # print(dvd_total)

    growth_rate_of_10y = (work_table['bookValuePerShare'][0]) / (work_table['bookValuePerShare'].last_valid_index())
    avg_growth_rate = growth_rate_of_10y ** (1 / 10)
    # print(avg_growth_rate)

    bvps_pred = (work_table['bookValuePerShare'][0]) * (avg_growth_rate ** 10)

    # print(work_table['bookValuePerShare'])

    data_intrinsic = bvps_pred + dvd_total
    # print(data_intrinsic)

    # data_intrinsic - latest_price  # Маржа безопасности

    # 'year_index' as the order of the year
    work_table['year_index'] = range(1, len(work_table) + 1)[::-1]
    # add intercept
    work_table['intercept'] = 1

    # print(work_table['year_index'])

    # modeling
    bvps_model = sm.OLS(work_table['bookValuePerShare'], work_table[['year_index', 'intercept']]).fit()

    bvps_pred_lr = bvps_model.predict([19, 1])[0]  # Прогнозируем балансовую стоимость через 10 лет
    # print(f'bvps_pred_lr через 10 лет: {bvps_pred_lr}')

    dvd_model = sm.OLS(work_table['Dividend per Share'][0:10].replace('', 0, regex=True).astype(float),
                       work_table[['year_index', 'intercept']]).fit()

    dvd_model.predict([19, 1])[0]  # Определить дивиденд через 10 лет

    # Дивиденды за 10 лет
    dvd_total_lr = 0
    current_year_order = 9
    for j in range(1, 11):
        dvd_new = dvd_model.predict([current_year_order + j, 1])[0]
        dvd_total_lr = dvd_total_lr + dvd_new / (1 + risk_free_rate) ** j
    # print(f'dvd_total_lr: {dvd_total_lr}')

    aapl_intrinsic_lr = dvd_total_lr + bvps_pred_lr
    # print(f'aapl_intrinsic_lr: {aapl_intrinsic_lr}')

    return round(aapl_intrinsic_lr, 2)